##二手车交易价格预测
https://tianchi.aliyun.com/competition/entrance/231784/rankingList

In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [38]:
train_data = pd.read_csv('./used_car_train.csv', sep=' ')
test_data = pd.read_csv('./used_car_testB.csv', sep=' ')
#print(train_data.info())
train_data.head()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,seller,offerType,creatDate,price,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,0.0,1046,0,0,20160404,1850,43.357796,3.966344,0.050257,2.159744,1.143786,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,-,4366,0,0,20160309,3600,45.305273,5.236112,0.137925,1.380657,-1.422165,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,0.0,2806,0,0,20160402,6222,45.978359,4.823792,1.319524,-0.998467,-0.996911,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,0.0,434,0,0,20160312,2400,45.687478,4.492574,-0.050616,0.883600,-2.228079,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,0.0,6977,0,0,20160313,5200,44.383511,2.031433,0.572169,-1.571239,2.246088,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482


In [39]:
train_data['regDate'] = train_data['regDate'].apply(lambda x: np.floor(x/10000))
train_data['creatDate'] = train_data['creatDate'].apply(lambda x: np.floor(x/100))

In [40]:
# 数据清洗
train_data.replace(to_replace='-', value=np.nan, inplace=True)
test_data.replace(to_replace='-', value=np.nan, inplace=True)
# 用中位数填充nan
train_data.fillna(train_data.median(), inplace=True)
test_data.fillna(test_data.median(), inplace=True)

tags = train_data.columns.tolist()
tags.remove('price')
tags.remove('SaleID')
tags.remove('name')
print(tags)

# 处理异常数据
#train_data['power'].describe()
train_data['power'][train_data['power']>600] = 600  # power字段：0~600

['regDate', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode', 'seller', 'offerType', 'creatDate', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [41]:
# 数据规范化
min_max_scalar = MinMaxScaler()
X_data = min_max_scalar.fit_transform(train_data[tags].values)
X_test = min_max_scalar.transform(test_data[tags].values)
y_data = train_data['price'].values

X_train, X_valid, y_train, y_valid = train_test_split(X_data, y_data, test_size=0.2)

In [42]:
from tensorflow import keras

# 模型搭建
model = keras.Sequential([
    keras.layers.Dense(250, activation='relu', input_shape=[len(tags)]),  # 输入的维度是特征的维度
    keras.layers.Dense(250, activation='relu'),
    keras.layers.Dense(250, activation='relu'),
    keras.layers.Dense(1)
])
# 定义损失函数，优化器
model.compile(loss='mean_absolute_error', optimizer='Adam')
# 训练
model.fit(X_train, y_train, batch_size=4096, epochs=150)

Epoch 1/150
29/29 [==============================] - 0s 5ms/step - loss: 5892.2446
Epoch 2/150
29/29 [==============================] - 0s 4ms/step - loss: 5229.9302
Epoch 3/150
29/29 [==============================] - 0s 4ms/step - loss: 4332.7559
Epoch 4/150
29/29 [==============================] - 0s 4ms/step - loss: 4139.2764
Epoch 5/150
29/29 [==============================] - 0s 3ms/step - loss: 3855.3762
Epoch 6/150
29/29 [==============================] - 0s 3ms/step - loss: 3254.6343
Epoch 7/150
29/29 [==============================] - 0s 3ms/step - loss: 2369.8958
Epoch 8/150
29/29 [==============================] - 0s 3ms/step - loss: 1802.8883
Epoch 9/150
29/29 [==============================] - 0s 3ms/step - loss: 1458.2809
Epoch 10/150
29/29 [==============================] - 0s 3ms/step - loss: 1289.2975
Epoch 11/150
29/29 [==============================] - 0s 3ms/step - loss: 1198.3018
Epoch 12/150
29/29 [==============================] - 0s 3ms/step - loss: 1138.6188
E

In [43]:
print('train set MAE:', mean_absolute_error(y_train, model.predict(X_train)))
print('validation set MAE:', mean_absolute_error(y_valid, model.predict(X_valid)))

train set MAE: 551.6996865911422
validation set MAE: 558.9854448679506


In [44]:
answer = pd.DataFrame()
answer['SaleID'] = test_data.SaleID
answer['price'] = model.predict(X_test)
answer.to_csv('./ans_nn.csv', index=False)